In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

lista_de_imoveis = []
pagina = 1

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

for pagina in range(1, 3201):
    pagina += 1
    url = f'https://www.wimoveis.com.br/imoveis-aluguel-distrito-federal-pagina-{pagina}.html'
    resposta = requests.get(url, headers=headers)
    conteudo = resposta.content

    site = BeautifulSoup(conteudo, 'html.parser')

    if not site.findAll('div', attrs={'class': 'sc-i1odl-0 dreQQz'}):
        print(f"Não há mais imóveis na página {pagina}. Finalizando a busca.")
        break
    
    # HTML do anúncio do imóvel
    imoveis = site.findAll('div', attrs={'class': 'sc-i1odl-0 dreQQz'})

    for imovel in imoveis:
        # Título do imóvel
        titulo = imovel.find('div', attrs={'class': 'sc-ge2uzh-0 eWOwnE postingAddress'})
        titulo_text = titulo.text if titulo else None

        # Subtítulo do imóvel
        subtitulo = imovel.find('h2', attrs={'data-qa': 'POSTING_CARD_LOCATION'})
        subtitulo_text = subtitulo.text.strip() if subtitulo else None

        # Descrição
        descricao = imovel.find('h3', attrs={'data-qa': 'POSTING_CARD_DESCRIPTION'})
        descricao_text = descricao.text if descricao else None
        
        # Link do imovel
        link = 'https://www.wimoveis.com.br' + imovel['data-to-posting']

        # Link logo imobiliaria 
        link_logo = imovel.find('img', attrs={'data-qa': 'POSTING_CARD_PUBLISHER'})    
            
        # Preco aluguel ou venda
        preco = imovel.find('div', attrs={'data-qa': 'POSTING_CARD_PRICE'})
        preco_text = preco.text if preco else None

        # Preço Condomínio
        preco_condominio = imovel.find('div', attrs={'data-qa': 'expensas'})
        preco_condominio_text = preco_condominio.text if preco_condominio else None   
            
        # Metro quadrado
        metro_area = imovel.find('h3', attrs={'data-qa': 'POSTING_CARD_FEATURES'})
        metro = metro_area.find('span') if metro_area else None
        metro_text = metro.text.replace('m²', '').strip() if metro else None

        # quartos, suíte, vagas
        quarto_suite_vaga = imovel.find('h3', attrs={'data-qa': 'POSTING_CARD_FEATURES'})
        lista = quarto_suite_vaga.text.split() if quarto_suite_vaga else None

        lista_de_imoveis.append([titulo_text, subtitulo_text, descricao_text, link, link_logo['src'] if link_logo else None, preco_text, preco_condominio_text , metro_text, lista])


        



df_imovel = pd.DataFrame(lista_de_imoveis, columns=['Título', 'Subtítulo', 'Descrição', 'Link', 'Link Logo Imobiliária', 'Preço', 'Preço Condomínio', 'Metro Quadrado', 'Metro, Quarto, Suite, Vaga'])
df_imovel.to_excel(r'C:\Users\galva\OneDrive\Documentos\GitHub\web-scrapping-com-python\teste_script\wimoveis_scrapping_aluguel.xlsx', index=False)
resposta

Não há mais imóveis na página 1191. Finalizando a busca.


<Response [500]>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url_base = 'https://www.wimoveis.com.br/imoveis-aluguel-distrito-federal-pagina-{}.html'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

sessao = requests.Session()
lista_de_imoveis = []
pagina = 1

while True:
    url = url_base.format(pagina)
    resposta = sessao.get(url, headers=headers)
    conteudo = resposta.content
    site = BeautifulSoup(conteudo, 'html.parser')
    
    imoveis = site.findAll('div', attrs={'data-qa': 'posting PROPERTY'})
    
    if not imoveis:
        break
    
    for imovel in imoveis:
        try:
            titulo = imovel.find('div', attrs={'class': 'sc-ge2uzh-0 eWOwnE postingAddress'})
            subtitulo = imovel.find('h2', attrs={'data-qa': 'POSTING_CARD_LOCATION'})
            descricao = imovel.find('h3', attrs={'data-qa': 'POSTING_CARD_DESCRIPTION'})
            link = 'https://www.wimoveis.com.br' + imovel['data-to-posting']
            preco = imovel.find('div', attrs={'data-qa': 'POSTING_CARD_PRICE'})
            preco_condominio = imovel.find('div', attrs={'data-qa': 'expensas'})
            metro_area = imovel.find('h3', attrs={'data-qa': 'POSTING_CARD_FEATURES'})
            metro = metro_area.find('span') if metro_area else None
            quarto_suite_vaga = imovel.find('h3', attrs={'data-qa': 'POSTING_CARD_FEATURES'})
            lista = quarto_suite_vaga.text.split() if quarto_suite_vaga else None
            link_logo = imovel.find('img', class_='postingCard__publisherLogo')['src']

            lista_de_imoveis.append([
                titulo.text.strip() if titulo else None,
                subtitulo.text.strip() if subtitulo else None,
                descricao.text.strip() if descricao else None,
                link,
                preco.text.strip() if preco else None,
                preco_condominio.text.strip() if preco_condominio else None,
                metro.text.strip() if metro_area else None,
                lista.strip() if lista else None,
                link_logo
            ])
        except Exception as e:
            print(f"Erro ao processar o imóvel na página {pagina}: {e}")
    
    pagina += 1

df_imovel = pd.DataFrame(lista_de_imoveis, columns=['Título', 'Subtítulo', 'Descrição', 'Link', 'Preço', 'Preço Condomínio', 'Metro Quadrado', 'Quartos', 'Suítes', 'Vagas', 'Link Logo Imobiliária'])
df_imovel.to_excel('wimoveis_scrapping_aluguel.xlsx', index=False)
